# Initial training

In [15]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd



path = 'smoke_detection_iot.csv'

data = pd.read_csv(path).drop(['Unnamed: 0', 'UTC', 'CNT'], axis=1)
Y_df = data['Fire Alarm']
X_df = data.drop('Fire Alarm', axis=1)

Y_test_raw = X_df.values
Y_raw = Y_df.values

X_train_raw, X_test_raw, Y_train_raw, Y_test_raw = train_test_split(Y_test_raw, Y_raw, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_raw = scaler.fit_transform(X_train_raw)
X_test_raw = scaler.transform(X_test_raw)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from alarmnetclass import AlarmNet
import torch
from torch import nn

alpha = 1e-2
epochs = 3000
device = 'cuda' if torch.cuda.is_available() else 'cpu'
full_model = AlarmNet(
    num_features=X_train_raw.shape[1],
    activation = nn.ReLU,
    hidden_layers=[64, 32]
    
).to(device)

X_train_raw_device = torch.tensor(X_train_raw).float().to(device)
X_test_raw_device = torch.tensor(X_test_raw).float().to(device)
Y_train_raw_device = torch.tensor(Y_train_raw).float().view(-1, 1).to(device)
Y_test_raw_device = torch.tensor(Y_test_raw).float().view(-1, 1).to(device)
full_model.train(epochs, X_train_raw_device, X_test_raw_device, Y_train_raw_device, Y_test_raw_device, alpha)
full_model.print_results()

Epoch 0 Loss: 0.6938216686248779
Epoch 500 Loss: 0.0029248378705233335
Epoch 1000 Loss: 0.0005927391466684639
Epoch 1500 Loss: 0.0002855720813386142
Epoch 2000 Loss: 0.00018974723934661597
Epoch 2500 Loss: 0.00012954707199241966
Accuracy: 0.9940124540954814
Precision: 0.9936383928571428
Recall: 0.9979822889810559
F1: 0.9958056037134388
Confusion_matrix:
[[3548   57]
 [  18 8903]]
Classification_report:               precision    recall  f1-score   support

         0.0       0.99      0.98      0.99      3605
         1.0       0.99      1.00      1.00      8921

    accuracy                           0.99     12526
   macro avg       0.99      0.99      0.99     12526
weighted avg       0.99      0.99      0.99     12526



# Feature Analysis
- Initially there are 15 features
- 3 are unusable
    - UTC Timestamp
    - CNT
    - Unnamed: 0
- 12 features are usable

In [17]:
import numpy as np



corr = np.abs(data.corr())
ranking = corr['Fire Alarm'].sort_values(ascending=False)[1:]
print(ranking)

Humidity[%]       0.399846
Raw Ethanol       0.340652
Pressure[hPa]     0.249797
TVOC[ppb]         0.214743
Temperature[C]    0.163902
NC0.5             0.128118
PM1.0             0.110552
Raw H2            0.107007
eCO2[ppm]         0.097006
PM2.5             0.084916
NC1.0             0.082828
NC2.5             0.057707
Name: Fire Alarm, dtype: float64


In [18]:
num_drops = 8
remaining_features = ranking.index[:-num_drops]


X_df_dropped = data[remaining_features]
print('Remaining features:', X_df_dropped.columns)
X_train_dropped, X_test_dropped, Y_train_dropped, Y_test_dropped = train_test_split(X_df_dropped.values, Y_raw, test_size=0.2, random_state=0)

X_train_dropped = scaler.fit_transform(X_train_dropped)
X_test_dropped = scaler.transform(X_test_dropped)


Remaining features: Index(['Humidity[%]', 'Raw Ethanol', 'Pressure[hPa]', 'TVOC[ppb]'], dtype='object')


In [21]:
dropped_model = AlarmNet(
    num_features=X_train_dropped.shape[1],
    activation = nn.ReLU,
    hidden_layers=[64, 32]
    
).to(device)

X_train_dropped_device = torch.tensor(X_train_dropped).float().to(device)
X_test_dropped_device = torch.tensor(X_test_dropped).float().to(device)
Y_train_dropped_device = torch.tensor(Y_train_dropped).float().view(-1, 1).to(device)
Y_test_dropped_device = torch.tensor(Y_test_dropped).float().view(-1, 1).to(device)

dropped_model.train(epochs, X_train_dropped_device, X_test_dropped_device, Y_train_dropped_device, Y_test_dropped_device, alpha)
dropped_model.print_results()

AlarmNet.compare_results(dropped_model.get_results(), full_model.get_results())

Epoch 0 Loss: 0.7010210752487183
Epoch 500 Loss: 0.01389069203287363
Epoch 1000 Loss: 0.049671173095703125
Epoch 1500 Loss: 0.046767473220825195
Epoch 2000 Loss: 0.04530363902449608
Epoch 2500 Loss: 0.0446198433637619
Accuracy: 0.9726169567300016
Precision: 0.9705946894886988
Recall: 0.9915928707543997
F1: 0.980981425006931
Confusion_matrix:
[[3337  268]
 [  75 8846]]
Classification_report:               precision    recall  f1-score   support

         0.0       0.98      0.93      0.95      3605
         1.0       0.97      0.99      0.98      8921

    accuracy                           0.97     12526
   macro avg       0.97      0.96      0.97     12526
weighted avg       0.97      0.97      0.97     12526

Comparing results:
accuracy: -2.1997865878683465 %
precision: -2.3741839531991786 %
recall: -0.644359032330994 %
f1: -1.5111579412834586 %


## Feature Analysis Results
- We can remove the bottom 8 features and have a model that only loses 0.1% precision, and even less for every other metric
- A 4 feature model is almost perfect
    - These features are:
        - Humidity
        - Raw Ethanol
        - Pressure
        - TVOC

# Error Handling
- The 12 initial features came from 4 sensors
    - Temp/Humidity
    - Pressure
    - Volatile Organic Compounds (CO2, Ethanol, H2, TVOC)
    - Particulate Matter (PM1, PM2.5, NC0.5, NC1, NC2.5)
- 3 of these sensors are redundant, with the exception being the PM sensor
    - This means that the features related to the PM sensor are twice as likely to be missing
- We can simulate a real world scenario by introducing error according to this distribution
- Note that the Particulate Matter sensor is not included in the 4-feature model.
    - To add redundancy to our model, we can add back the most correlated feature from the PM sensor
    - This feature is PM0.5



In [ ]:

# Add back the most correlated PM feature, so that all 4 sensors are used
# PM0.5 is the most correlated PM feature, with index 5
remaining_features_2 = list(remaining_features)
remaining_features_2.append(ranking.index[5])
X_5f = data[remaining_features_2]
X_train_5f, X_test_5f, Y_train_5f, Y_test_5f = train_test_split(X_5f.values, Y_raw, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_5f = scaler.fit_transform(X_train_5f)
X_test_5f = scaler.transform(X_test_5f)


In [22]:

dropped_model_5 = AlarmNet(
    num_features=X_train_5f.shape[1],  # Update to match the new input dimensions
    activation = nn.ReLU,
    hidden_layers=[64, 32]
    
).to(device)
epochs = 3000
alpha = 1e-2

X_train_5f_device = torch.tensor(X_train_5f).float().to(device)
X_test_5f_device = torch.tensor(X_test_5f).float().to(device)
Y_train_5f_device = torch.tensor(Y_train_raw).float().view(-1, 1).to(device)
Y_test_5f_device = torch.tensor(Y_test_raw).float().view(-1, 1).to(device)

dropped_model_5.train(epochs, X_train_5f_device, X_test_5f_device, Y_train_5f_device, Y_test_5f_device, alpha)

dropped_model_5_results = dropped_model_5.get_results()
dropped_model_5.print_results()

AlarmNet.compare_results(dropped_model_5_results, full_model.get_results())


Epoch 0 Loss: 0.6826237440109253
Epoch 500 Loss: 0.011827047914266586
Epoch 1000 Loss: 0.054410360753536224
Epoch 1500 Loss: 0.05195501819252968
Epoch 2000 Loss: 0.050904519855976105
Epoch 2500 Loss: 0.0504881776869297
Accuracy: 0.999041992655277
Precision: 0.9987683350128765
Recall: 0.999887904943392
F1: 0.9993278064082456
Confusion_matrix:
[[3594   11]
 [   1 8920]]
Classification_report:               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3605
         1.0       1.00      1.00      1.00      8921

    accuracy                           1.00     12526
   macro avg       1.00      1.00      1.00     12526
weighted avg       1.00      1.00      1.00     12526

Comparing results:
accuracy: 0.5034361515103059 %
precision: 0.5136268317584943 %
recall: 0.19058295964125666 %
f1: 0.35245718894445044 %


In [24]:
import random
from sklearn.impute import SimpleImputer

# Introduce sensor errors

VOC_features = [
    'TVOC[ppb]',
    'eCO2[ppm]',
    'Raw H2',
    'Raw Ethanol'
]

PM_features = [
    'PM1.0',
    'PM2.5',
    'NC0.5',
    'NC1.0',
    'NC2.5'
]

th_features = [
    'Temperature[C]',
    'Humidity[%]'
]
pressure_features = [
    'Pressure[hPa]'
]

# For each measurement, each sensor has this chance of introducing an error
error_chance = 0.2

# The PM sensor is twice as likely to fail due to lack of redundancy
chances = [error_chance, error_chance*2, error_chance, error_chance]
sensors = [VOC_features, PM_features, th_features, pressure_features]

error_mask = np.ones(X_5f.shape)

# for i, row in error_mask:
#     errored_features = []
#     for j, sensor in enumerate(sensors):
#         sensor_error = random.random() < chances[j]
#         if sensor_error:
#             errored_features.extend(sensor)
#     errored_features = [feature for feature in errored_features if feature in X_error.columns]
#     if errored_features:
#         for feature in errored_features:
#             error_mask[i][X_error.columns.get_loc(feature)] = np.nan
#         print(i, error_mask[i])

X_error_np = X_5f.values.copy()
for i, datapoint in enumerate(X_5f.values):
    errored_features = []
    for j, sensor in enumerate(sensors):
        sensor_error = random.random() < chances[j]
        if sensor_error:
            errored_features.extend(sensor)
    errored_features = [feature for feature in errored_features if feature in X_5f.columns]
    if errored_features:
        for feature in errored_features:
            X_error_np[i][X_5f.columns.get_loc(feature)] = np.nan
        # print(i, X_error_np[i])

X_train_error, X_test_error, Y_train_error, Y_test_error = train_test_split(X_error_np, Y_raw, test_size=0.2, random_state=0)

## Imputation
- Replace each errored value with the mean of that feature from the training data

In [25]:
imputer = SimpleImputer(strategy='mean')
#imputer.fit(X_5f.values)
imputer.fit(X_train_error)
X_impute_train = imputer.transform(X_train_error)
X_impute_test = imputer.transform(X_test_error)

X_impute_train = scaler.fit_transform(X_impute_train)
X_impute_test = scaler.transform(X_impute_test)

In [26]:
imputed_model = AlarmNet(
    num_features=X_impute_train.shape[1],
    activation = nn.ReLU,
    hidden_layers=[128, 256, 128]
    
).to(device)

X_impute_train_device = torch.tensor(X_impute_train).float().to(device)
X_impute_test_device = torch.tensor(X_impute_test).float().to(device)
Y_train_impute_device = torch.tensor(Y_train_error).float().view(-1, 1).to(device)
Y_test_impute_device = torch.tensor(Y_test_error).float().view(-1, 1).to(device)

imputed_model.train(
    epochs= 16000,
    X_train = X_impute_train_device,
    X_test = X_impute_test_device,
    Y_train = Y_train_impute_device,
    Y_test = Y_test_impute_device,
    alpha= 1e-4,
    loss_fn = nn.BCELoss(),
    optimizer = torch.optim.Adam
)
imputed_model.print_results()
AlarmNet.compare_results(imputed_model.get_results(), dropped_model_5.get_results())

Epoch 0 Loss: 0.6883925795555115
Epoch 500 Loss: 0.17090661823749542
Epoch 1000 Loss: 0.13784246146678925
Epoch 1500 Loss: 0.1114770919084549
Epoch 2000 Loss: 0.07624808698892593
Epoch 2500 Loss: 0.05974220111966133
Epoch 3000 Loss: 0.05209113284945488
Epoch 3500 Loss: 0.04784805327653885
Epoch 4000 Loss: 0.0450618751347065
Epoch 4500 Loss: 0.043051548302173615
Epoch 5000 Loss: 0.041324738413095474
Epoch 5500 Loss: 0.03987447917461395
Epoch 6000 Loss: 0.038674090057611465
Epoch 6500 Loss: 0.03717140853404999
Epoch 7000 Loss: 0.03625910356640816
Epoch 7500 Loss: 0.035581715404987335
Epoch 8000 Loss: 0.035002559423446655
Epoch 8500 Loss: 0.03459536284208298
Epoch 9000 Loss: 0.03415282443165779
Epoch 9500 Loss: 0.03381981700658798
Epoch 10000 Loss: 0.033510755747556686
Epoch 10500 Loss: 0.03325588256120682
Epoch 11000 Loss: 0.033162716776132584
Epoch 11500 Loss: 0.03287045657634735
Epoch 12000 Loss: 0.03272054344415665
Epoch 12500 Loss: 0.032776907086372375
Epoch 13000 Loss: 0.03255430236

## Error Modes
- Current features:
    - Humidity
    - Raw Ethanol
    - Pressure
    - TVOC
    - NC0.5
- Sensors:
    - Humidity
    - Pressure
    - Raw Ethanol/TVOC
    - NC0.5
- The model should be able to handle missing data in the case where at most 3 sensors have failed, because 4 failed sensors means no data
    - 1 failed sensor = 4c1 = 4
    - 2 failed sensors = 4c2 = 6
    - 3 failed sensors = 4c3 = 4
    - Total = 14
## Ensemble Training
- We can train 14 models that can predict the missing data for each error mode
- In the case of error, we select the model that corresponds to the error mode and use it to predict the missing data
- Then use the main model to predict the target
### Indexing Ensemble
- Columns should be rearranged according to sensors
    - Humidity, Pressure, NC0.5, Ethanol, TVOC, 
- The error mode can be represented as a 4-bit value
    - 0b0000 = No error
    - 0b0001 = Ethanol/TVOC Error
    - 0b0010 = NC0.5 Error
    - 0b0100 = Pressure error
    - 0b1000 = Humidity error
- A 5-bit value can represent which features are missing
    - 0b00000 = No error
    - 0b00011 = Ethanol/TVOC Error
    - 0b00100 = NC0.5 Error
    - 0b01000 = Pressure error
    - 0b10000 = Humidity error
- We can convert from the 5-bit value to the 4-bit value with a simple shift right operation

### Model Table
- Store the models with an array
- The index of the model is the error mode
- Model 15 will always predict 1, because if all sensors have failed the worst should be assumed for safety
- Model 0 will be the standard model trained on a full dataset
- The rest of the arrays will be trained on the data with the corresponding error mode
### New Model Type
- A new model class will be created that will predict the missing values, construct the repaired dataset, call the standard model, and return the result
### Ensemble Class
- This new class will hold the model table and the standard model
- It will be responsible for constructing the model address, calling the correct model, and returning the result



    

